# ChatBot Mental Health

# 1. Connect Dengan Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#open data
import json
with open('/content/intentsindo.json') as file:
  data = json.load(file)

print(data)

# 2. Download Library

In [ ]:
import nltk
import pickle
import numpy as np
import tensorflow as tf
import random
!pip install tensorflow
!pip install keras
!pip install keras-models
!pip install nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

from keras.models import load_model

# mendefinisikan variabel
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

#membaca file json dan menyimpan ke dalam variabel intents
data_file = open("/content/intentsindo.json").read()
intents = json.loads(data_file)

# 3. Data Pre-Processing

* Tokenization
* Membah dokumen dari corpus
* Menambahkan Ke kelas

In [ ]:
for intent in intents['intents']:
    if 'patterns' not in intent:
        intent['patterns'] = []
    for pattern in intent['patterns']:
        # tokenisasi kata
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # menambah dokumen ke dalam corpus
        documents.append((w, intent['tag']))
    # menambah ke dalam list kelas
    if intent['tag'] not in classes:
        classes.append(intent['tag'])
print("words", words)
print("documents",documents)
print("classes",classes)

In [ ]:

# membuat lematization
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# mengurutkan kelas
classes = sorted(list(set(classes)))

# documents 
print(len(documents), "documents")

# intents sebagai kelas
print(len(classes), "classes", classes)

# vocab
print(len(words), "unique lemmatized words", words)

# membuat file pickle 
pickle.dump(words,open('texts.pkl','wb')) 
pickle.dump(classes,open('label.pkl','wb'))

# 4. Data Training

In [ ]:
training = []

output_empty = [0]* len(classes)

for doc in documents:
  #instal bag of word
  bag = []

  #list untuk patterns
  patterns_words = doc[0]

  patterns_words = [lemmatizer.lemmatize(word.lower()) for word in patterns_words]

  for w in words:
    bag.append(1) if w in patterns_words else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Data Telah di Training")

In [ ]:
print(len(train_x))
print(len(train_y))

In [ ]:
#contoh data yang telah dirubah menjadi vektor numerik dalam bentuk array
train_x

# 5. Modeling Neural Network

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Reshape
import matplotlib.pyplot as plt
#ini library dari tensorflow untuk agar kecepatan melatih model dapat disesuaikan

model = Sequential()
model.add(Dense(261, input_shape=(len(train_x[0]),), activation='relu'))

#menambah lapisan input layer #shape karna data berupa array
model.add(Dropout(0.5))
#menambahkan lapisan dropout agar tidak overfitting
model.add(Dense(64, activation='relu'))
#hidden layer
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
#output layer

rmsprop = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])
#algoritma deep learning agar model tidak loss function
#loss fubction untuk multiclass klasification

#memasang dan menyimpan model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) 

plt.plot(hist.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [ ]:
model.summary()

# 6. Repeat -> mengulangi langkah agar dapat merespon terus menerus

In [ ]:
#simpan dalam file model
from tensorflow.keras.models import load_model
model = load_model('/content/chatbot.h5')
intents = json.loads(open("/content/intentsindo.json").read())
words = pickle.load(open('/content/texts.pkl','rb'))
classes = pickle.load(open('/content/label.pkl','rb'))

In [ ]:
print(np.array(train_x).shape)

In [ ]:
words

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# memprediksi kelas dari kalimat yang ditraining sebelumnya
def clean_up_sentence(sentence):

    # pisahkan data dan dalam bentuk array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True, return_bag=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
        return np.array(bag)

# memprediksi kelas dari kalimat yang ditraining sebelumnya
max_sequence_length = 406
def predict_class(sentence, model, words, classes, max_sequence_length):
    p = bow(sentence, words, show_details=False, return_bag=True)
    p = np.array([p])
    padded_input_data = pad_sequences(p, maxlen=max_sequence_length, padding='post', truncating='post')
    return_list = []
    res = model.predict(padded_input_data)[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [ ]:
print(predict_class( "stres",  model, words, classes, max_sequence_length))

In [ ]:
# berfungsi untuk mendapatkan respons dari model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    result = None  # initialize result variable
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    if result is None:  # handle case where no matching intent is found
        result = "I'm sorry, I didn't understand what you said."
    return result

def chatbot_response(text):
    # memprediksi kelas intent dari kalimat yang diinput
    ints = predict_class(text, model, words, classes, max_sequence_length)

    # mencari respon terbaik berdasarkan kelas intent
    res = getResponse(ints, intents)

    return res

In [ ]:
# berfungsi untuk mendapatkan respons dari model

def getResponse(ints, intents_json):
  try:
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    result = None  # initialize result variable
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    if result is None:  # handle case where no matching intent is found
        result = "Maaf, tampaknya saya tidak mengerti maksud anda. Saya akan mempelajari maksud anda."
  except IndexError:
        result = "Maaf, tampaknya saya tidak mengerti maksud anda. Saya akan mempelajari maksud anda"
  return result


In [ ]:
# bot akan berakhir jika mengetik end

def start_chat():
    print("Bot: This is Sophie! Your Personal Assistant.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

# 7. Coba BotChat

In [ ]:
# start the chat bot
start_chat()